In [1]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 10.2 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [3]:
import os
from pyngrok import ngrok

In [4]:
ngrok.kill()

In [5]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://jeanice-unmonetary-suzanne.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://jeanice-unmonetary-suzanne.ngrok-free.dev


True

In [6]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [7]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [8]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學（Minghsin University of Science and Technology），簡稱明新科大，是一所位於臺灣新竹縣新豐鄉的私立科技大學。 學校坐擁新竹科學園區與新竹工業區的豐富產業資源，以「產業大學」為辦學定位。

**歷史沿革**
明新科技大學的創校可追溯至1966年3月1日，當時以「明新工業專科學校」立案，初期設有機械、土木、工業管理三科，並招收三百名新生。 隨後陸續增設電機工程科、化學工程科等。 1993年，學校更名為「私立明新工商專科學校」。 1997年7月，奉教育部核准改制為「明新技術學院」並附設專科部，期間增設了旅館事業管理、幼兒保育、休閒事業管理、老人服務事業管理等科系。 2002年8月，明新獲教育部核准升格為「明新科技大學」。 2018年12月，學校更名為「明新學校財團法人明新科技大學」。

**學術組織**
明新科技大學現設有六個學院，包括：半導體學院、工程學院、管理學院、民生學院、人文與設計學院以及共同教育學院。 這些學院涵蓋了20個學系、2個學位學程（含1個博士學位學程）及11個碩士班。 2021年，學校整合原隸屬於工學院的電機工程系、電子工程系、化學工程與材料科技系、光電工程系，成立了半導體學院。 2022年10月，更獲教育部核准通過「半導體科技博士學位學程」，這是該校成立以來的第一個博士班。

**辦學理念與特色**
明新科技大學的校名「明新」取自《大學》「在明明德，在新民，在止於至善」的精義，旨在闡揚人類與生俱來的德性與情操，培育學子高尚品德、專業學問與優良技術。 學校的校訓為「堅毅、求新、創造」。

學校在辦學上具有多項亮點：
*   **國際化**：明新科大擁有全國第一的國際學生人數，學生來自超過16個國家地區，並與百餘所大學姐妹校進行交流合作，提供短期交換、雙聯學位等機會。
*   **半導體人才培育**：學校是新竹科學園區科技人才培育的搖籃，特別著重於半導體人才的培育。 設有約1100坪的「半導體產業設備廠務與檢測人才培育基地」，旨在打造「Mini TSMC」的實務教學環境。
*   **AI教育**：學校致力於將AI技術全面融入教學，將程式設計與人工智慧概論列為大一必修課，並開發「APEC國際會議模擬混合實境教室」等，培養學生的AI敏銳力與數位素養。
*   **產學連結**：學校以「產業大學」為定位

In [9]:
result2 = stateful_query("校長是誰？")
print(result2)

明新科技大學現任校長為**呂明峯教授**。 他於2025年2月1日正式上任，成為明新科技大學的第11任校長。

呂明峯校長在業界擁有豐富的經驗，學術涵養深厚。 他曾任電子工程系主任、研發長、工學院院長、半導體學院院長以及產學長。 在其擔任半導體學院院長期間，他開辦了全國首見的「2+2N封裝測試產業精英專班」，並爭取預算打造全台第一座半導體封裝測試類產線。 此外，他也積極推動成立半導體學院和半導體科技博士學位學程，使明新科大在半導體教育領域聲譽卓著。


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Jan/2026 16:38:33] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"Ufe0037d5ecf7ada870601eb0bc93c619","events":[]}
BODY:  {"destination":"Ufe0037d5ecf7ada870601eb0bc93c619","events":[{"type":"message","message":{"type":"text","id":"594675560866381859","quoteToken":"z4L6T2v8U1ISmtSNslG7VHbLqEcgdictEhR0yiYxBUzb0umkVyBOuXqkmC_PBdxlQVQ-h8nYdBPNnJIadGU58ivxFp6uW2xCLusBz8BJ7P4ucZE_8NFaJS_ZY_g9bS3J1LabAXOvz-jYZNu2S-IQ6Q","markAsReadToken":"QTuYiRZMePRrHSqFEJujz0PXVY6zLrahPmjIvD1ZiipzI4UwYxwzYh_4Sk_xuQgjL3RFcGlV34L3u0CHO4EExMam-hqWWrapQ68lbMdZKE5ilBzLVYQWp3of8QsZujk73tMhegpUokCR4P7ZTfZajBzkVtjAYSNZW3qFO_qUOPzFo862Xbz-YsWAKhPxejJgcYku9HYngoNFlirsnNUXXw","text":"AI 介紹一下明新科大(五十字以内)"},"webhookEventId":"01KDX6V4S2Y937KBEDRJN8KQAN","deliveryContext":{"isRedelivery":false},"timestamp":1767285559628,"source":{"type":"user","userId":"U708a8161d116dd816aab6ebb0ad0157c"},"replyToken":"feb36cee897440ff9491ecee6cbda318","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [01/Jan/2026 16:39:22] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"Ufe0037d5ecf7ada870601eb0bc93c619","events":[{"type":"message","message":{"type":"text","id":"594675600074735685","quoteToken":"khN3o5KNX26nw5PP6sdX7vn1cKHRrkCjRFqacMiQCwQXr9kPEd4y0o6bD_fHJzZ6q7F7YpNXv0HZ3H7k_Hi-jUeKE0Fc6PH6dP1HnNrBp5EJrnZuwUB4MfP_IQ-zHehrfLoR0LkMNd1D3ZapzqmzUg","markAsReadToken":"dDN83pTQNKSXICrp91WFlKsq6YDMBtZdfBdXNeIWeyus-Lhnineaq3qr__Xq-0DveR8Jd0hMhbHUOGKqls5NWQFivAKLhNO2FG5RViBn2PjcDu6X3awDit_dpSxhlyA9DzhmqdzF8xWHlIQUODmDEJ1peOnVowTACKmCU82SaCOVhCrf8rQr8-0ZilauIbzduepu7cXme1txCookFaD9OQ","text":"AI 校長是誰?"},"webhookEventId":"01KDX6VVDVPEJCTTTPA2V1NG22","deliveryContext":{"isRedelivery":false},"timestamp":1767285582999,"source":{"type":"user","userId":"U708a8161d116dd816aab6ebb0ad0157c"},"replyToken":"4430f9040a3445b7afd6415fa85f06c1","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [01/Jan/2026 16:39:45] "POST / HTTP/1.1" 200 -
